Run this in a terminal to make sure the shared folder is mounted (only at the beginning when booting up the VM.)

    sudo mount -t vboxsf -o uid=$UID,gid=$(id -g) simulation_results ~/Documents/sf_simulation_results/

In [6]:
%%bash
cd ../chaste_build/
make CartilageSheetSimulation

[  5%] Built target timekeeper
[ 10%] Built target chaste_global
[ 10%] Built target chaste_linalg
[ 15%] Built target chaste_io
[ 21%] Built target chaste_ode
[ 42%] Built target chaste_mesh
[ 47%] Built target chaste_pde
[ 94%] Built target chaste_cell_based
Scanning dependencies of target chaste_project_cartilage
[ 94%] Building CXX object projects/cartilage/CMakeFiles/chaste_project_cartilage.dir/src/NodeBasedMesenchymalCondensation.cpp.o
[ 94%] Linking CXX shared library libchaste_project_cartilage.so
[100%] Built target chaste_project_cartilage
Scanning dependencies of target CartilageSheetSimulation
[100%] Building CXX object projects/cartilage/apps/CMakeFiles/CartilageSheetSimulation.dir/src/CartilageSheetSimulation.cpp.o
[100%] Linking CXX executable CartilageSheetSimulation
[100%] Built target CartilageSheetSimulation


In [3]:
import sys
sys.path.append('analysis')

import numpy as np
import time 
import matplotlib.pyplot as plt
import distribute_executable as de
import sheet_metrics as sm

plt.style.use('seaborn-whitegrid')
#plt.style.use('tableau-colorblind10')
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (6.75, 5),
          'lines.linewidth': 3.0,
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large',
         'font.size': 12}
plt.rcParams.update(params)

# Let's try and see
We set
- lower and upper rigid boundaries
- lower and upper perichondrial layer
- chondrocyte layers in between, nicely stacked at cartesian coordinates, filling the space between the rigid boundaries
- repulsion only
- oriented cell division
- clonal patch size maximum to 6 cells 

## First without scaling

Data archive: 
- 6 random seeds (0--5), patch size limit of 6 cells:
      nice -n 19 /home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation --output-dir exp-intercalation/20200331-173524/ --S 0 --sbt --T 80.0 --sw 8 --sd 8 --sh 6 --pu 1 --pl 1 --nb 2 --mu 0.0 --A 0.25 --F pwq --mu_R 80.0 --p 0.1 --u 5.0 --cg 

## With scaling

In [ ]:
- 6 random seeds (0--5):
    

In [7]:
output_directory = 'exp-intercalation/' + time.strftime("%Y%m%d-%H%M%S") +'/'
flags = ' --sbt' # synchronized birth times
flags = flags + ' --T 80.0' # final time 80h
flags = flags + ' --sw 8' # 10 cells in x direction
flags = flags + ' --sd 8' # 20 cells in y direction
flags = flags + ' --sh 6' # 6 cells in z direction
flags = flags + ' --pu 1' # 1 layer perichondrial cells above
flags = flags + ' --pl 1' # 1 layer perichondrial cells below
flags = flags + ' --nb 2' # both an upper and a lower rigid boundary
flags = flags + ' --mu 0.0' # adhesive spring stiffness set to 0 -> repulsion only
flags = flags + ' --A 0.25' # 25% of stem cells activated
flags = flags + ' --F pwq' # PWQ force
flags = flags + ' --mu_R 80.0' # set repulsive spring stiffness to 80.0
flags = flags + ' --p 0.1' # set perturbation to 0.1
flags = flags + ' --psl 4'
flags = flags + ' --u 3.5' # set distance between boundaries to 3.5
flags = flags + ' --cg ' # use a Cartesian grid for initial placement
flags = flags + ' --sc 1.25' # scale initial sheet to generate more space


#flags = flags + ' --rdd' #random division directions

executable='/home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation'

In [8]:
de.run_simulations_multiple_random_seeds(output_directory, flags=flags, number_of_simulations=6, executable=executable)

nice -n 19 /home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation --output-dir exp-intercalation/20200331-173524/ --S 0 --sbt --T 80.0 --sw 8 --sd 8 --sh 6 --pu 1 --pl 1 --nb 2 --mu 0.0 --A 0.25 --F pwq --mu_R 80.0 --p 0.1 --u 5.0 --cg 
nice -n 19 /home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation --output-dir exp-intercalation/20200331-173524/ --S 1 --sbt --T 80.0 --sw 8 --sd 8 --sh 6 --pu 1 --pl 1 --nb 2 --mu 0.0 --A 0.25 --F pwq --mu_R 80.0 --p 0.1 --u 5.0 --cg 
nice -n 19 /home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation --output-dir exp-intercalation/20200331-173524/ --S 2 --sbt --T 80.0 --sw 8 --sd 8 --sh 6 --pu 1 --pl 1 --nb 2 --mu 0.0 --A 0.25 --F pwq --mu_R 80.0 --p 0.1 --u 5.0 --cg 
nice -n 19 /home/kubuntu1804/Documents/chaste_build/projects/cartilage/apps/CartilageSheetSimulation --output-dir exp-intercalation/20200331-173524/ --S 3 --sbt --T 80.0 --sw 8 --sd 8 

In [9]:
de.run_postprocessing_multiple_random_seeds(6, output_directory)

['/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/0/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/1/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/2/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/3/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/4/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/5/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/6/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/7/results_from_time_0/', '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/8/results_from_time_0/', '/home/ku

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FileNotFoundError: [Errno 2] No such file or directory: '/home/kubuntu1804/Documents/sf_simulation_results/exp-intercalation/20200331-173524/6/results_from_time_0/results.viznodes'